In [1]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "7"
sys.path.append('../') 

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

import random
from demo.ConfSeq import get_ConfSeq_pair_from_mol,get_mol_from_ConfSeq_pair,randomize_mol
import copy
from tqdm import tqdm
from rdkit.Chem import rdmolops, rdchem
from tqdm.contrib.concurrent import process_map  

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from transformers import BartForConditionalGeneration, BartConfig
from collections import OrderedDict
import torch
from torch import nn

import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

from rdkit.ML.Scoring import Scoring
import pandas as pd

/home/xiongjiacheng/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
vocab = [chr(i) for i in range(33, 127)] 

for i in range(-180,180):
    vocab.append('<'+str(i)+'>')

vocab.append('<mask>')
vocab.append('<unk>')
vocab.append('<sos>')
vocab.append('<eos>')
vocab.append('<pad>')


config = BartConfig()

config.pad_token_id = vocab.index('<pad>')
config.eos_token_id = vocab.index('<eos>')
config.sos_token_id = vocab.index('<sos>')
config.forced_eos_token_id = None
config.encoder_layers = 6
config.encoder_attention_heads = 8
config.decoder_layers = 0
config.decoder_attention_heads = 0
config.d_model = 256
# config.share_embeddings = True
config.vocab_size = len(vocab)


vocab_dict = {char: idx for idx, char in enumerate(vocab)}
bart = BartForConditionalGeneration(config = config )


class CustomBartEncoder(nn.Module):
    def __init__(self, bart):
        super().__init__()
    
        self.bart_model = bart 
        
    def forward(self, input_ids, attention_mask=None):
        outputs = self.bart_model.model.encoder(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state

bart_encoder_model  = CustomBartEncoder(bart)

In [ ]:
def rm_invalid_chirality(mol):
    mol = copy.deepcopy(mol)
    rings = rdmolops.GetSymmSSSR(mol)

    atom_in_rings_count = {}

    for ring in rings:
        for atom_idx in ring:
            if atom_idx not in atom_in_rings_count:
                atom_in_rings_count[atom_idx] = 0
            atom_in_rings_count[atom_idx] += 1

    atoms_in_3_rings = [atom for atom, count in atom_in_rings_count.items() if count == 3]

    for atom_idx in atoms_in_3_rings:
        atom = mol.GetAtomWithIdx(atom_idx)
        atom.SetChiralTag(rdchem.ChiralType.CHI_UNSPECIFIED)

    return mol

In [5]:
def get_ConfSeq(query_mol):

    if query_mol != None:
        try:
            query_mol = rm_invalid_chirality(query_mol)
            query_mol = randomize_mol(query_mol)
            Chem.MolToSmiles(query_mol)    
            query_mol = Chem.RenumberAtoms(query_mol, eval(query_mol.GetProp('_smilesAtomOutputOrder'))) 
            Chem.MolToSmiles(query_mol,canonical = False)
            query_mol = Chem.RenumberAtoms(query_mol, eval(query_mol.GetProp('_smilesAtomOutputOrder'))) 
            in_smiles,TD_smiles = get_ConfSeq_pair_from_mol(query_mol)
            TD_smiles = TD_smiles.replace('<180>','<-180>')
            
        except:
            in_smiles,TD_smiles = '',''            
    else:
        in_smiles,TD_smiles = '',''
        
    return TD_smiles

In [6]:
def get_embedding_for_seq(i):
    int = [vocab_dict[i] for i in i.split(' ')] 
    int = torch.Tensor([int]).long()
    embed = bart_encoder_model(int).mean(dim=1).tolist()[0]
    return embed

In [ ]:
class TensorDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.Tensor(self.data[idx]).long()

def collate_fn(batch):
    data = batch  
    padded_data = pad_sequence(data, batch_first=True, padding_value = vocab.index('<pad>'))  
    return padded_data


def mean_pooling(last_hidden_state, attention_mask):

    attention_mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size())

    sum_embeddings = (last_hidden_state * attention_mask).sum(dim=1)
    sum_mask = attention_mask.sum(dim=1)

    pooled_output = sum_embeddings / (sum_mask + 1e-8)  
    return pooled_output

In [8]:
device = torch.device('cuda')

In [ ]:
checkpoint = torch.load('./checkpoints/model_epoch_1.pth', map_location='cpu')   


new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    if k.startswith('module.'):
        new_state_dict[k[7:]] = v
    else:
        new_state_dict[k] = v

bart_encoder_model.load_state_dict(new_state_dict)

bart_encoder_model.eval()



lis = os.listdir('./data/DUDE/all/')
lis = [i for i in lis if '.txt' not in i]

for name in lis[:]:
    os.makedirs('./data/DUDE/all_result/{}/'.format(name), exist_ok=True)
    
    query_mol = Chem.MolFromMolFile('./data/DUDE/all/{}/crystal_ligand.sdf'.format(name))
    query_mol_atom_num = query_mol.GetNumAtoms()
    
    suppl = Chem.SDMolSupplier('./data/DUDE/all/{}/actives.sdf'.format(name))

    actives_mols = [mol for mol in suppl if mol is not None]
    actives_names = [i.GetProp('_Name') for i in actives_mols]
    actives_mols_atom_num = [i.GetNumAtoms() for i in actives_mols]
    
    suppl = Chem.SDMolSupplier('./data/DUDE/all/{}/decoys.sdf'.format(name))

    decoys_mols = [mol for mol in suppl if mol is not None]
    decoys_names = [i.GetProp('_Name') for i in decoys_mols]
    decoys_mols_atom_num = [i.GetNumAtoms() for i in decoys_mols]
    


    actives_seqs = process_map(get_ConfSeq, tqdm(actives_mols), max_workers = 32)
    print('actives_seqs length:',len(actives_seqs))
    
    decoys_seqs = process_map(get_ConfSeq, tqdm(decoys_mols), max_workers = 32)
    print('decoys_seqs length:',len(decoys_seqs))

    with open('./data/DUDE/all_result/{}/actives_seqs.txt'.format(name),'w+') as f:
        f.write('\n'.join(actives_seqs))

    with open('./data/DUDE/all_result/{}/decoys_seqs.txt'.format(name),'w+') as f:
        f.write('\n'.join(decoys_seqs))


    with open('./data/DUDE/all_result/{}/actives_seqs.txt'.format(name),'r') as f:
        content = f.read()
    active_seqs = content.split('\n')   
    active_seqs = [i if i != '' else '*' for i in actives_seqs]
    
    
    with open('./data/DUDE/all_result/{}/decoys_seqs.txt'.format(name),'r') as f:
        content = f.read()
    decoys_seqs = content.split('\n') 
    decoys_seqs = [i if i != '' else '*' for i in decoys_seqs]
    bart_encoder_model.eval()
    

    device = torch.device('cuda')
    bart_encoder_model = bart_encoder_model.to(device)

    active_ints = [[vocab_dict[i] for i in seq.split(' ')] for seq in actives_seqs]
    active_dataset = TensorDataset(active_ints)
    active_dataloader = DataLoader(active_dataset, batch_size=32,collate_fn=collate_fn,shuffle = False)
    active_embeds = []
    
    with torch.no_grad():  
    
        for input_ids in active_dataloader:
            input_ids = input_ids.to(device)
            attention_mask = (input_ids != vocab.index('<pad>')).long() 
            
            output = bart_encoder_model(input_ids,attention_mask)
            pooled_output = mean_pooling(output, attention_mask)
            active_embeds.append(pooled_output)
    
    active_embeds = torch.cat(tuple(active_embeds), dim=0).to('cpu')

    inactive_ints = [[vocab_dict[i] for i in seq.split(' ')] for seq in decoys_seqs]
    inactive_dataset = TensorDataset(inactive_ints)
    inactive_dataloader = DataLoader(inactive_dataset, batch_size=32,collate_fn=collate_fn,shuffle = False)
    inactive_embeds = []
    
    with torch.no_grad():  
    
        for input_ids in inactive_dataloader:
            input_ids = input_ids.to(device)
            attention_mask = (input_ids != vocab.index('<pad>')).long() 
   
            output = bart_encoder_model(input_ids,attention_mask)
            pooled_output = mean_pooling(output, attention_mask)
            inactive_embeds.append(pooled_output)
    
    inactive_embeds = torch.cat(tuple(inactive_embeds), dim=0).to('cpu')


    fingerprints = np.vstack([active_embeds,inactive_embeds])
    labels = np.array([1] * len(active_embeds) + [0] * len(inactive_embeds))  

    mols_atom_num = actives_mols_atom_num + decoys_mols_atom_num
    
    
    bart_encoder_model.to(torch.device('cpu'))
    query_embeds = get_embedding_for_seq(get_ConfSeq(query_mol))
    query_embeds = torch.Tensor(query_embeds)
    euclidean_distance = torch.sqrt(torch.sum((query_embeds - fingerprints ) ** 2,dim = -1))
    similarity_score = 1 / (1 + euclidean_distance)

    mols_atom_num = np.array(mols_atom_num)
    factor = 2 * mols_atom_num/ (query_mol_atom_num + mols_atom_num)
    similarity_score = similarity_score * factor

    

    df = pd.DataFrame({"Name": actives_names + decoys_names,
                       "Score": similarity_score.tolist(),
                       "label": labels.tolist()})
    df.to_csv('./data/DUDE/all_result/{}/my_model_result.csv'.format(name), index=False)

    if torch.is_tensor(similarity_score):
        similarity_score_np = similarity_score.cpu().numpy()
    else:
        similarity_score_np = similarity_score

    sort_idx = np.argsort(-similarity_score_np)
    sorted_scores = similarity_score_np[sort_idx]
    sorted_labels = labels[sort_idx]

    sorted_labels_bool = sorted_labels.astype(bool)

    score_data = np.column_stack((sorted_scores, sorted_labels_bool))

    auc_rdkit = Scoring.CalcAUC(score_data, col=1)

    bedroc_rdkit = Scoring.CalcBEDROC(score_data, col=1, alpha=80.5)

    ef_1 = Scoring.CalcEnrichment(score_data, col=1, fractions=[0.01])[0]
    ef_5 = Scoring.CalcEnrichment(score_data, col=1, fractions=[0.05])[0]
    

    row = {
        "Name": name,
        "AUC": round(auc_rdkit, 4),
        "BEDROC": round(bedroc_rdkit, 4),
        "EF5%": round(ef_5, 4),
        "EF1%": round(ef_1, 4)
    }
    

    csv_path = './DUDE_evaluation_result.csv'
    if not os.path.exists(csv_path):
        df = pd.DataFrame([row])
        df.to_csv(csv_path, index=False)
    else:
        df = pd.DataFrame([row])
        df.to_csv(csv_path, mode='a', index=False, header=False)

100%|█████████████████████████████████████████| 485/485 [00:00<00:00, 1462.78it/s]


actives_seqs length: 485


  0%|                                                   | 0/18219 [00:00<?, ?it/s]/tmp/ipykernel_2731164/2398411433.py:44: TqdmWarning: Iterable length 18219 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  decoys_seqs = process_map(get_ConfSeq, tqdm(decoys_mols), max_workers = 32)
100%|█████████████████████████████████████| 18219/18219 [00:07<00:00, 2320.81it/s]


decoys_seqs length: 18219


100%|█████████████████████████████████████████| 185/185 [00:00<00:00, 1211.22it/s]


...

decoys_seqs length: 4626


100%|█████████████████████████████████████████| 844/844 [00:00<00:00, 1482.67it/s]


actives_seqs length: 844


  0%|                                                   | 0/32015 [00:00<?, ?it/s]/tmp/ipykernel_2731164/2398411433.py:44: TqdmWarning: Iterable length 32015 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  decoys_seqs = process_map(get_ConfSeq, tqdm(decoys_mols), max_workers = 32)
100%|█████████████████████████████████████| 32015/32015 [00:13<00:00, 2424.28it/s]


decoys_seqs length: 32015
